In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb


In [ ]:
vocab_size = 10000     # use only the 10,000 most common words
maxlen = 200           # each review will be truncated or padded to 200 words

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

#Each review is already converted into a list of word IDs (integers).
#Example: [1, 14, 22, 16, 43, ...] → represents a sentence.


In [ ]:
print(X_train[0])


In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

print("Decoded Review:", decode_review(X_train[0]))
print("Label:", y_train[0])

#IMDB encodes words as numbers. This function converts them back to words so we can see the actual review.

In [ ]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

#Not all reviews have the same length.
#Padding ensures all sequences are exactly maxlen words long so they fit into one tensor.

["The movie was great"] → [the, movie, was, great, <pad.>, <pad.>, ...]


---



In [ ]:
model = Sequential()

# Step 1: Word Embedding
#Embedding: Converts each word index → 128-dim vector (learned automatically).
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=maxlen))

# Step 2: Recurrent Layer
#SimpleRNN: Reads one word at a time, keeps a “memory” of what it has seen before.
model.add(SimpleRNN(128, activation='tanh'))

# Step 3: Output Layer
#Dense: Outputs a single value between 0 and 1 (positive or negative sentiment).
model.add(Dense(1, activation='sigmoid'))


In [ ]:
from tensorflow.keras.optimizers import SGD

opt = SGD(learning_rate=0.1, momentum=0.1)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2)


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


## Concept Recap

| Concept | Meaning |
|----------|----------|
| **RNN** | A network that has memory — it processes data sequentially (like text or time series). |
| **Hidden state** | Internal memory that carries information from previous steps. |
| **Limitation** | SimpleRNN struggles with long sentences due to vanishing gradients (LSTM solves that). |

---



##  Advantages and Disadvantages of RNN

###  Advantages
- Can handle **sequential data** (like text, speech, or time series).  
- **Remembers previous inputs** through hidden states.  
- Works well for **language modeling** and **sequence prediction** tasks.  
- **Same weights** used across time steps — reduces model complexity.  

###  Disadvantages
- **Vanishing gradient problem** — struggles with long sequences.  
- **Training is slow** due to sequential processing.  
- **Difficult to parallelize**, unlike CNNs or Transformers.  
- Can **forget long-term dependencies** in long data sequences.  


# **Project 1 RNN (Fill missing words)**

In [ ]:
# ==============================================================
# 🧠 Simple RNN Text Prediction Example
# ==============================================================

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Small dataset (sentences)
sentences = [
    "hard work leads to success",
    "practice makes you better",
    "never stop learning new things",
    "dream big and stay focused",
    "success comes from consistency",
    "believe in yourself and work hard",
    "great things take time",
    "failure is the first step to success",
    "push yourself because no one else will do it for you",
    "discipline beats motivation every single time"
]

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1

# Create training data (input sequences)
input_sequences = []
for line in sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_seq_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre'))

# Split inputs (X) and labels (y)
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Convert labels to one-hot
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build RNN model
model = Sequential([
    Embedding(total_words, 10, input_length=max_seq_len-1),
    SimpleRNN(50),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=300, verbose=0)

print("✅ Model trained successfully!")


In [ ]:
# ==============================================================
# 💬 Try your own input phrase
# ==============================================================

def predict_next_word(seed_text, model, tokenizer, max_seq_len):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == np.argmax(predicted):
            output_word = word
            break
    return output_word

# 🔹 Try your sentence here:
seed_text = "practice makes you"
next_word = predict_next_word(seed_text, model, tokenizer, max_seq_len)
print(f"🧠 Input: '{seed_text}' → Predicted next word: '{next_word}'")


# **project example 2 (Mini RNN Translator (English → Spanish) — Working Version)**

In [ ]:


import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ✨ Small dataset for demo (English → Spanish)
data = {
    "hello": "hola",
    "how are you": "como estas",
    "good morning": "buenos dias",
    "good night": "buenas noches",
    "thank you": "gracias",
    "i love you": "te amo",
    "see you later": "hasta luego",
    "my name is": "mi nombre es",
    "what is your name": "como te llamas",
    "have a nice day": "que tengas un buen dia"
}

# Prepare source (English) and target (Spanish)
eng_texts = list(data.keys())
spa_texts = ["<start> " + txt + " <end>" for txt in data.values()]

# Tokenize both languages
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(eng_texts)
spa_tokenizer = Tokenizer()
spa_tokenizer.fit_on_texts(spa_texts)

eng_seq = eng_tokenizer.texts_to_sequences(eng_texts)
spa_seq = spa_tokenizer.texts_to_sequences(spa_texts)

max_eng_len = max(len(seq) for seq in eng_seq)
max_spa_len = max(len(seq) for seq in spa_seq)

eng_seq = pad_sequences(eng_seq, maxlen=max_eng_len, padding='post')
spa_seq = pad_sequences(spa_seq, maxlen=max_spa_len, padding='post')

eng_vocab = len(eng_tokenizer.word_index) + 1
spa_vocab = len(spa_tokenizer.word_index) + 1

spa_input = spa_seq[:, :-1]
spa_target = spa_seq[:, 1:]

# ------------------------------
# Build Seq2Seq Model (Encoder-Decoder)
# ------------------------------
latent_dim = 64

# Encoder
encoder_inputs = Input(shape=(max_eng_len,))
enc_emb = Embedding(eng_vocab, 64)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_spa_len - 1,))
dec_emb_layer = Embedding(spa_vocab, 64)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(spa_vocab, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Full training model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit([eng_seq, spa_input], np.expand_dims(spa_target, -1), epochs=300, verbose=0)

print("✅ Model trained successfully!")

# ------------------------------
# Create Inference Models
# ------------------------------
# Encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
dec_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=dec_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model([decoder_inputs] + dec_states_inputs, [decoder_outputs2] + decoder_states2)

# ------------------------------
# Translation Function
# ------------------------------
def translate(sentence):
    seq = eng_tokenizer.texts_to_sequences([sentence])
    seq = pad_sequences(seq, maxlen=max_eng_len, padding='post')
    states_value = encoder_model.predict(seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = spa_tokenizer.word_index['start']
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = ''
        for word, index in spa_tokenizer.word_index.items():
            if index == sampled_token_index:
                sampled_word = word
                break
        if sampled_word == 'end' or len(decoded_sentence.split()) > max_spa_len:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()


In [ ]:
# Try it interactively 🚀
input_sentence = "good morning"
print(f"🌍 English: {input_sentence}")
print(f"🇪🇸 Spanish: {translate(input_sentence)}")
